In [2]:
import pandas as pd
import json
import numpy as np

In [20]:
data = json.load(open('purchase_data.json', 'r'))
heroes = json.dumps(data)
df = pd.read_json(heroes)
#df['Price'] = df['Price'].map('${:.2f}'.format)
df.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


### Total players

In [4]:
players = df['SN'].unique().shape[0]
print("total players: {}".format(players))

total players: 573


### Purchasing Analysis

In [5]:
#number of unique items
unique_items = df['Item ID'].unique().shape[0]
unique_items

183

In [9]:
#average purchase price
avg_price = '${}'.format(round(df["Price"].mean(),2))
avg_price

'$2.93'

In [10]:
#total Number purchases
purchase_count = df['Item Name'].count()
purchase_count

780

In [11]:
#total revenue
total_revenue = '${}'.format(round(df["Price"].sum(),2))
total_revenue

'$2286.33'

In [13]:
purch_analysis = pd.DataFrame({
    "Item Count": [unique_items],
    "Average Purchase Price": [avg_price],
    "Total Number of Purchases": [purchase_count],
    "Total Revenue": [total_revenue]
})
purch_analysis

,Average Purchase Price,Item Count,Total Number of Purchases,Total Revenue
0,$2.93,183,780,$2286.33


### Gender Demographics

In [17]:
#percentage and count of players by gender
gender = df.groupby('Gender')['SN'].nunique().reset_index()
gender = gender.rename(columns={'SN': 'Gender_Count'})
total_players = gender.Gender_Count.sum()
gender['Percentage of Players'] = (gender['Gender_Count']/total_players*100)
gender['Percentage of Players'] = gender['Percentage of Players'].map('{:.2f}%'.format)
gender

,Gender,Gender_Count,Percentage of Players
0,Female,100,17.45%
1,Male,465,81.15%
2,Other / Non-Disclosed,8,1.40%


### Purchasing Analysis (Gender)

In [21]:
#purchase count, avg purch price, total purch value, normalized total
gender_purchase_analysis = df.groupby('Gender').Price.agg(['count', 'mean', 'sum']).reset_index()
gender_purchase_analysis = gender_purchase_analysis.rename(columns={'count': 'Purchase Count', 'mean': 'Avg Purchase Price','sum': 'Total Purchase Value'})
gender_purchase_analysis['Normalized Totals'] = gender_purchase_analysis['Total Purchase Value'] / gender['Gender_Count']
gender_purchase_analysis['Avg Purchase Price'] = gender_purchase_analysis['Avg Purchase Price'].map('${:.2f}'.format)
gender_purchase_analysis['Total Purchase Value'] = gender_purchase_analysis['Total Purchase Value'].map('${:,.2f}'.format)
gender_purchase_analysis['Normalized Totals'] = gender_purchase_analysis['Normalized Totals'].map('${:.2f}'.format)
gender_purchase_analysis

,Gender,Purchase Count,Avg Purchase Price,Total Purchase Value,Normalized Totals
0,Female,136,$2.82,$382.91,$3.83
1,Male,633,$2.95,"$1,867.68",$4.02
2,Other / Non-Disclosed,11,$3.25,$35.74,$4.47


### Age demographics

In [17]:
#create bins 
bins = [0, 10, 14, 19, 24, 29, 34, 39, 100]
age_group_labels = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39','40+']
df['Age Groups'] = pd.cut(df['Age'], bins, labels=age_group_labels)
age_counts = (df.groupby("Age Groups"))['SN'].nunique().reset_index()
age_counts = age_counts.rename(columns={'SN': 'Total Count'})
age_counts

,Age Groups,Total Count
0,<10,22
1,10-14,20
2,15-19,100
3,20-24,259
4,25-29,87
5,30-34,47
6,35-39,27
7,40+,11


In [18]:
#purchase count, avg purchase price, total purchase value, normalized totals
age_purch_data = df.groupby('Age Groups').Price.agg(['count', 'mean', 'sum']).reset_index()
age_purch_data = age_purch_data.rename(columns={'count': 'Purchase Count', 'mean': 'Avg Purchase Price','sum': 'Total Purchase Value'})
age_purch_data['Normalized Totals'] = age_purch_data['Total Purchase Value'] / age_counts['Total Count']
age_purch_data['Avg Purchase Price'] = age_purch_data['Avg Purchase Price'].map('${:.2f}'.format)
age_purch_data['Total Purchase Value'] = age_purch_data['Total Purchase Value'].map('${:.2f}'.format)
age_purch_data['Normalized Totals'] = age_purch_data['Normalized Totals'].map('${:.2f}'.format)

age_purch_data

,Age Groups,Purchase Count,Avg Purchase Price,Total Purchase Value,Normalized Totals
0,<10,32,$3.02,$96.62,$4.39
1,10-14,31,$2.70,$83.79,$4.19
2,15-19,133,$2.91,$386.42,$3.86
3,20-24,336,$2.91,$978.77,$3.78
4,25-29,125,$2.96,$370.33,$4.26
5,30-34,64,$3.08,$197.25,$4.20
6,35-39,42,$2.84,$119.40,$4.42
7,40+,17,$3.16,$53.75,$4.89


### Top Spenders
by purchase value

In [34]:
#SN, Purchase Count, Avg Purch price, Total Purch Value
df_top_cust = df.groupby('SN').Price.agg(['count', 'mean', 'sum']).nlargest(5, "sum").reset_index()
df_top_cust = df_top_cust.rename(columns={'count': 'Purchase Count', 'mean': 'Avg Purchase Price','sum': 'Total Purchase Value'})
df_top_cust['Avg Purchase Price'] = df_top_cust['Avg Purchase Price'].map('${:.2f}'.format)
df_top_cust['Total Purchase Value'] = df_top_cust['Total Purchase Value'].map('${:.2f}'.format)
df_top_cust

,SN,Purchase Count,Avg Purchase Price,Total Purchase Value
0,Undirrala66,5,$3.41,$17.06
1,Saedue76,4,$3.39,$13.56
2,Mindimnya67,4,$3.18,$12.74
3,Haellysu29,3,$4.24,$12.73
4,Eoda93,3,$3.86,$11.58


### Most Popular item
by purchase count

In [32]:
#Item ID, Item Name, Purchase Count, Item Price
df_top_prod = df.groupby(['Item ID', 'Item Name', 'Price']).Price.agg(['count', 'mean', 'sum']).nlargest(5, "count").reset_index()
df_top_prod = df_top_prod.rename(columns={'count': 'Purchase Count', 'mean': 'Average Price','sum': 'Total Purchase Value'})
df_top_prod['Price'] = df_top_prod['Price'].map('${:.2f}'.format)
df_top_prod['Total Purchase Value'] = df_top_prod['Total Purchase Value'].map('${:.2f}'.format)
del df_top_prod["Average Price"]
df_top_prod

,Item ID,Item Name,Price,Purchase Count,Total Purchase Value
0,39,"Betrayal, Whisper of Grieving Widows",$2.35,11,$25.85
1,84,Arcane Gem,$2.23,11,$24.53
2,13,Serenity,$1.49,9,$13.41
3,31,Trickster,$2.07,9,$18.63
4,34,Retribution Axe,$4.14,9,$37.26


### Most Profitable
by purchase value

In [33]:
#Item ID, Item Name, Purchase Count, Item Price, Total Purchase Value
df_top_profit = df.groupby(['Item Name', 'Item ID', 'Price']).Price.agg(['count', 'mean', 'sum']).nlargest(5, "sum").reset_index()
df_top_profit = df_top_profit.rename(columns={'count': 'Purchase Count', 'mean': 'Average Price','sum': 'Total Purchase Value'})
df_top_profit['Total Purchase Value'] = df_top_profit['Total Purchase Value'].map('${:.2f}'.format)
df_top_profit['Price'] = df_top_profit['Price'].map('${:.2f}'.format)
del df_top_profit['Average Price']
df_top_profit

,Item Name,Item ID,Price,Purchase Count,Total Purchase Value
0,Retribution Axe,34,$4.14,9,$37.26
1,Spectral Diamond Doomblade,115,$4.25,7,$29.75
2,Orenmir,32,$4.95,6,$29.70
3,Singed Scalpel,103,$4.87,6,$29.22
4,"Splitter, Foe Of Subtlety",107,$3.61,8,$28.88


### observations
1. 15-30 year old males are largely the dominant consumers of this video game.
2. No single product is a revenue driver for them. The most popular item only sold 11 times.
3. Even with 183 items available, no one purchased more than five items so they must be to a very targeted audience